In [1]:
from clustering import *
from downloaddata import *
from portfolio_and_performance import *
from trees import *
import cPickle as pickle

In [2]:
import gc
picklename = open('pickle/MST_WRDS_LedoitWolf_spaced_1985.p', 'rb')
gc.disable()
trees = pickle.load(picklename)
gc.enable()
picklename.close()

In [3]:
sorteddates = sorted(trees.keys(), key=lambda d: map(int, d.split('-')))
IGtrees = {}
for key in sorteddates:
    IGtrees[key] = NXtoIG(trees[key])

In [5]:
gc.disable()
picklename = open('pickle/LedoitWolf_clustersNewman.p', 'rb')
clustersNewman = pickle.load(picklename) 
picklename.close()
picklename = open('pickle/LedoitWolf_clustersClausetNewman.p', 'rb')
clustersClausetNewman = pickle.load(picklename) 
picklename.close()
picklename = open('pickle/LedoitWolf_IGclustersNewman.p', 'rb')
IGclustersNewman = pickle.load(picklename) 
picklename.close()
picklename = open('pickle/LedoitWolf_IGclustersClausetNewman.p', 'rb')
IGclustersClausetNewman = pickle.load(picklename) 
picklename.close()
gc.enable()

In [6]:
clustersNewman, IGclustersNewman = construct_clusters(IGtrees, method='Newman')
clustersClausetNewman, IGclustersClausetNewman = construct_clusters(IGtrees, method="ClausetNewman")

In [4]:
window = 100
space = 10
thresh = 95
filename = "SP500_from_1985.csv"
enddate='2017-06-28'
startdate='1985-01-02'

In [8]:
pickle.dump(clustersNewman,open("pickle/LedoitWolf_clustersNewman.p",'wb'))
pickle.dump(IGclustersNewman,open("pickle/LedoitWolf_IGclustersNewman.p",'wb'))
pickle.dump(clustersClausetNewman,open("pickle/LedoitWolf_clustersClausetNewman.p",'wb'))
pickle.dump(IGclustersClausetNewman,open("pickle/LedoitWolf_IGclustersClausetNewman.p",'wb'))

In [9]:
NewmanUniverse = {}
ClausetNewmanUniverse = {}
for measure in ['degree', 'closeness', 'betweenness']:
    NewmanUniverse[measure] = clustering_universe(trees, clustersNewman, measure, quantile=0)
    ClausetNewmanUniverse[measure] = clustering_universe(trees, clustersClausetNewman, 
                                                         measure, quantile=0)
universes = {'Newman': NewmanUniverse, 'ClausetNewman': ClausetNewmanUniverse}

In [6]:
universes = pickle.load(open("pickle/universes_unshrunk.p", "rb"))

In [10]:
pickle.dump(universes, open("pickle/universes_LedoitWolf.p", "wb"))

In [7]:
price = importdata("data/WRDS/SP500_price_1985.csv").replace(0,np.nan).ffill()
ret = importdata("data/WRDS/SP500_ret_1985.csv").fillna(0)
div = importdata("data/WRDS/SP500_div_1985.csv").fillna(0)
cpr = importdata("data/WRDS/SP500_cpr_1985.csv").replace(0,np.nan).ffill()

In [8]:
shrinkage = 'LedoitWolf'

In [13]:
performance_weighted, performance_unweighted = \
    benchmark_performance(price, ret, div, cpr, thresh=thresh,
                          universes=universes['Newman']['betweenness'],
                          window=window, shrinkage=shrinkage)
performance_comparison_weighted = {'benchmark': performance_weighted}
performance_comparison_unweighted = {'benchmark': performance_unweighted}

In [14]:
pickle.dump(performance_comparison_unweighted, open("pickle/performance_LedoitWolf_unweighted.p", "wb"))
pickle.dump(performance_comparison_weighted, open("pickle/performance_LedoitWolf_weighted.p", "wb"))

In [16]:
import datetime

for method in ['Newman', 'ClausetNewman']:
    performance_comparison_unweighted[method] = {}
    for measure in ['degree', 'closeness', 'betweenness']:
        print "doing " + method + " " + measure + ": " + str(datetime.datetime.now().time())
        temp = clustering_performance(price, ret, div, cpr, thresh=thresh, universes=universes[method][measure],
                                      weighted='FALSE', window=window, shrinkage=shrinkage)
        performance_comparison_unweighted[method][measure] = pd.DataFrame.from_dict(temp)
        performance_comparison_unweighted[method][measure]['benchmark'] = \
        performance_comparison_unweighted['benchmark'][
            'benchmark']
        performance_comparison_unweighted[method][measure] = performance_comparison_unweighted[method][measure][[
            'benchmark', 'central', 'peripheral']]
        print "done" + ": " + str(datetime.datetime.now().time())
for method in ['Newman', 'ClausetNewman']:
    performance_comparison_weighted[method] = {}
    for measure in ['degree', 'closeness', 'betweenness']:
        print "doing " + method + " " + measure + ": " + str(datetime.datetime.now().time())
        temp = clustering_performance(price, ret, div, cpr, thresh=thresh, universes=universes[method][measure],
                                      weighted='TRUE', window=window, shrinkage=shrinkage)
        performance_comparison_weighted[method][measure] = pd.DataFrame.from_dict(temp)
        performance_comparison_weighted[method][measure]['benchmark'] = performance_comparison_weighted['benchmark'][
            'benchmark']
        performance_comparison_weighted[method][measure] = performance_comparison_weighted[method][measure][[
            'benchmark', 'central', 'peripheral']]
        print "done" + ": " + str(datetime.datetime.now().time())

doing Newman degree: 22:19:50.405490


done: 22:21:31.079253
doing Newman closeness: 22:21:31.079425


done: 22:23:07.833255
doing Newman betweenness: 22:23:07.833428


done: 22:24:48.885623
doing ClausetNewman degree: 22:24:48.885805


done: 22:26:31.841682
doing ClausetNewman closeness: 22:26:31.841771


...

done: 22:32:48.405656
doing Newman betweenness: 22:32:48.405778


done: 22:34:26.712006
doing ClausetNewman degree: 22:34:26.712240


done: 22:36:02.210496
doing ClausetNewman closeness: 22:36:02.210590


done: 22:37:35.975841
doing ClausetNewman betweenness: 22:37:35.975936


done: 22:39:13.903046


In [17]:
pickle.dump(performance_comparison_unweighted, open("pickle/performance_LedoitWolf_unweighted.p", "wb"))
pickle.dump(performance_comparison_weighted, open("pickle/performance_LedoitWolf_weighted.p", "wb"))

In [9]:
performance_comparison_unweighted = pickle.load(open("pickle/performance_LedoitWolf_unweighted.p", "rb"))
performance_comparison_weighted = pickle.load(open("pickle/performance_LedoitWolf_weighted.p", "rb"))

In [16]:
noclusteruniverse_0 = {}
performance_comparison_unweighted['nocluster'] = {}
for c_measure in ['degree', 'closeness', 'betweenness']:
    print "doing " + "nocluster" + " " + c_measure+ ": " + str(datetime.datetime.now().time())
    noclusteruniverse_0[c_measure] = no_cluster_universe(trees=trees, c_measure=c_measure, quantile=0.25)
    performance_comparison_unweighted['nocluster'][c_measure] = pd.DataFrame.from_dict(
        clustering_performance(price, ret, div, cpr, thresh=thresh, universes=noclusteruniverse_0[c_measure],
                               weighted='FALSE', window=window, shrinkage=shrinkage))
    print "done"+ ": " + str(datetime.datetime.now().time())

doing nocluster degree: 23:54:17.460329


done: 23:55:59.040036
doing nocluster closeness: 23:55:59.040276


done: 00:00:27.424770
doing nocluster betweenness: 00:00:27.424866


done: 00:06:24.512399


In [ ]:
#single nocluster universe, unweighted
single_noclusteruniverse = {}
performance_comparison_unweighted['single_nocluster'] = {}
for c_measure in ['degree', 'closeness', 'betweenness']:
    print "doing " + "single_cluster" + " " + c_measure+ ": " + str(datetime.datetime.now().time())
    single_noclusteruniverse[c_measure] = no_cluster_universe(trees=trees, c_measure=c_measure, quantile=0)
    performance_comparison_unweighted['single_nocluster'][c_measure] = pd.DataFrame.from_dict(
        clustering_performance(price, ret, div, cpr, thresh=thresh, universes=single_noclusteruniverse[c_measure],
                               weighted='FALSE', window=window, shrinkage=shrinkage))
    print "done"+ ": " + str(datetime.datetime.now().time())

doing nocluster degree: 23:54:17.460329


done: 23:55:59.040036
doing nocluster closeness: 23:55:59.040276


done: 00:00:27.424770
doing nocluster betweenness: 00:00:27.424866


done: 00:06:24.512399


In [21]:
performance_comparison_unweighted['nocluster']

{'betweenness':                central  peripheral
 1985-01-03    1.000000    1.000000
 1985-01-04    0.995082    0.995058
 1985-01-07    0.998035    0.997704
 1985-01-08    0.996313    0.998800
 1985-01-09    1.003200    1.002832
 1985-01-10    1.027403    1.013280
 1985-01-11    1.029110    1.017498
 1985-01-14    1.046369    1.029620
 1985-01-15    1.051848    1.036875
 1985-01-16    1.060597    1.044616
 1985-01-17    1.058849    1.046682
 1985-01-18    1.063521    1.050442
 1985-01-21    1.085002    1.065406
 1985-01-22    1.086880    1.066976
 1985-01-23    1.096091    1.077863
 1985-01-24    1.099517    1.077980
 1985-01-25    1.108385    1.082635
 1985-01-28    1.114873    1.086225
 1985-01-29    1.123468    1.095195
 1985-01-30    1.131333    1.100767
 1985-01-31    1.127355    1.100607
 1985-02-01    1.120269    1.097783
 1985-02-04    1.134249    1.103879
 1985-02-05    1.138206    1.107410
 1985-02-06    1.143931    1.111024
 1985-02-07    1.155940    1.118586
 1985-02-08  

In [22]:
import datetime

#no cluster universe, weighted--------------
performance_comparison_weighted['nocluster'] = {}
for c_measure in ['degree', 'closeness', 'betweenness']:
    print "doing " + "nocluster" + " " + c_measure+ ": " + str(datetime.datetime.now().time())
    performance_comparison_weighted['nocluster'][c_measure] = pd.DataFrame.from_dict(
        clustering_performance(price, ret, div, cpr, thresh=thresh, universes=noclusteruniverse_0[c_measure],
                               weighted='TRUE', window=window, shrinkage=shrinkage))
    print "done"+ ": " + str(datetime.datetime.now().time())

    #single nocluster universe, weighted
performance_comparison_weighted['single_nocluster'] = {}
for c_measure in ['degree', 'closeness', 'betweenness']:
    print "doing " + "single_cluster" + " " + c_measure+ ": " + str(datetime.datetime.now().time())
    performance_comparison_weighted['single_nocluster'][c_measure] = pd.DataFrame.from_dict(
        clustering_performance(price, ret, div, cpr, thresh=thresh, universes=single_noclusteruniverse[c_measure],
                               weighted='TRUE', window=window, shrinkage=shrinkage))
    print "done"+ ": " + str(datetime.datetime.now().time())

doing nocluster degree: 00:08:09.268729


done: 00:09:54.131108
doing nocluster closeness: 00:09:54.131207


done: 00:11:39.544558
doing nocluster betweenness: 00:11:39.544652


done: 00:13:22.754371
doing single_cluster degree: 00:13:22.754732


done: 00:14:56.619014
doing single_cluster closeness: 00:14:56.619138


done: 00:16:30.275133
doing single_cluster betweenness: 00:16:30.275316


done: 00:18:06.393760


In [10]:
single_noclusteruniverse = pickle.load(open("pickle/single_nocluster_universe_LedoitWolf.p", "rb"))
noclusteruniverse_0 = pickle.load(open("pickle/nocluster_universe_LedoitWolf.p", "rb"))

In [23]:
pickle.dump(performance_comparison_unweighted, open("pickle/performance_LedoitWolf_unweighted.p", "wb"))

In [24]:
pickle.dump(performance_comparison_weighted, open("pickle/performance_LedoitWolf_weighted.p", "wb"))

In [25]:
pickle.dump(single_noclusteruniverse, open("pickle/single_nocluster_universe_LedoitWolf.p", "wb"))
pickle.dump(noclusteruniverse_0, open("pickle/nocluster_universe_LedoitWolf.p", "wb"))

In [26]:
performance_LedoitWolf_unweighted = pickle.load(open("pickle/performance_LedoitWolf_unweighted.p", "rb"))
performance_LedoitWolf_weighted = pickle.load(open("pickle/performance_LedoitWolf_weighted.p", "rb"))
performance_unshrunk_unweighted = pickle.load(open("pickle/performance_unshunk_unweighted.p", "rb"))
performance_unshrunk_weighted = pickle.load(open("pickle/performance_unshunk_weighted.p", "rb"))

In [27]:
final_result = {'None': {'weighted': performance_unshrunk_weighted,
                         'unweighted': performance_unshrunk_unweighted},
                'LedoitWolf': {'weighted': performance_LedoitWolf_weighted,
                               'unweighted': performance_LedoitWolf_unweighted}
                }

In [28]:
pickle.dump(final_result, open("pickle/final_result.p", "wb"))

In [29]:
import matplotlib as mpl
mpl.is_interactive()
from matplotlib import pyplot as plt
plt.ioff()

In [30]:
import matplotlib.pyplot as plt

%matplotlib
for shrinkage in ['None','LedoitWolf']:
    for weighted in ['weighted','unweighted']:
        for method in final_result[shrinkage][weighted].keys():
            if method != 'benchmark':
                for measure in ['betweenness','degree','closeness']:
                    fig = plt.figure()
                    final_result[shrinkage][weighted][method][measure].plot(figsize=(20, 8))
                    fig.text(.1,.1,shrinkage+'_'+weighted+'_'+method+'_'+measure)
                    plt.savefig("Graphs/"+shrinkage+'_'+weighted+'_'+method+'_'+measure+".pdf", bbox_inches='tight')
            else:
                final_result[shrinkage][weighted][method].plot(figsize=(20, 8))
                plt.savefig("Graphs/"+shrinkage+'_'+weighted+'_'+method+'_'+".pdf", bbox_inches='tight')

Using matplotlib backend: Qt5Agg


/home/xuxiao2695/anaconda2/envs/Neduv/lib/python2.7/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [31]:
for shrinkage in ['None','LedoitWolf']:
    for weighted in ['weighted','unweighted']:
        for method in final_result[shrinkage][weighted].keys():
            if method != 'benchmark':
                for measure in ['betweenness','degree','closeness']:
                    for port in ['central','peripheral']:
                        print("%s: %f" % (shrinkage+'_'+weighted+'_'+method+'_'+measure+'_'+port
                                          , sharpe_ratio(
                            final_result[shrinkage][weighted][method][measure][port],0.01)))
            else:
                print("%s: %f" % (shrinkage+'_'+weighted+'_'+method
                                          , sharpe_ratio(
                            final_result[shrinkage][weighted][method]['benchmark'],0.01)))

None_weighted_Newman_betweenness_central: 0.712797
None_weighted_Newman_betweenness_peripheral: 1.015834
None_weighted_Newman_degree_central: 0.717881
None_weighted_Newman_degree_peripheral: 0.919497
None_weighted_Newman_closeness_central: 0.793827
None_weighted_Newman_closeness_peripheral: 0.909553
None_weighted_benchmark: 0.903382
None_weighted_ClausetNewman_betweenness_central: 0.506679
None_weighted_ClausetNewman_betweenness_peripheral: 1.020761
None_weighted_ClausetNewman_degree_central: 0.572815
None_weighted_ClausetNewman_degree_peripheral: 0.927211
None_weighted_ClausetNewman_closeness_central: 0.617496
None_weighted_ClausetNewman_closeness_peripheral: 0.910924
None_weighted_nocluster_betweenness_central: 0.673821
None_weighted_nocluster_betweenness_peripheral: 1.018595
None_weighted_nocluster_degree_central: 0.673454


None_weighted_nocluster_degree_peripheral: 0.929432
None_weighted_nocluster_closeness_central: 0.694232
None_weighted_nocluster_closeness_peripheral: 0.886595
None_weighted_single_nocluster_betweenness_central: 0.394447
None_weighted_single_nocluster_betweenness_peripheral: 1.018595
None_weighted_single_nocluster_degree_central: 0.481476
None_weighted_single_nocluster_degree_peripheral: 0.878712
None_weighted_single_nocluster_closeness_central: 0.391425
None_weighted_single_nocluster_closeness_peripheral: 0.537708
None_unweighted_Newman_betweenness_central: 0.671289
None_unweighted_Newman_betweenness_peripheral: 0.887636
None_unweighted_Newman_degree_central: 0.732724
None_unweighted_Newman_degree_peripheral: 1.002596
None_unweighted_Newman_closeness_central: 0.740025
None_unweighted_Newman_closeness_peripheral: 0.981584
None_unweighted_benchmark: 0.848655
None_unweighted_ClausetNewman_betweenness_central: 0.690064
None_unweighted_ClausetNewman_betweenness_peripheral: 0.892267
None_un